In [1]:
import pandas as pd
import numpy as np

In [2]:
print("Cargando archivos originales...")
in_store_df = pd.read_excel(
    '/home/juandi/Documents/TOM_JETSON/Data/cleaned_data_for_klaviyo.xlsx', engine='openpyxl'
)
online_store_df = pd.read_csv(
    '/home/juandi/Documents/TOM_JETSON/Data/customers_export.csv', low_memory=False
)

print(f"Tamaño de datos in-store: {in_store_df.shape}")
print(f"Tamaño de datos online: {online_store_df.shape}")

print("Renombrando columnas...")
in_store_df.rename(columns={
    'Name': 'Full Name',
    'Customer: Billing Address City': 'City',
    'Customer: Billing Address State': 'State',
    'PhoneNumber': 'Phone',
    'Email': 'Email',
    'Source Sales: # of Tickets': 'Total Orders',
    'Source Sales: Net Sales': 'Total Spent'
}, inplace=True)

online_store_df.rename(columns={
    'First Name': 'First Name',
    'Last Name': 'Last Name',
    'Email': 'Email',
    'Default Address City': 'City',
    'Default Address Province Code': 'State',
    'Total Spent': 'Total Spent',
    'Total Orders': 'Total Orders'
}, inplace=True)

online_store_df['Full Name'] = online_store_df['First Name'] + ' ' + online_store_df['Last Name']

in_store_df['Source'] = 'In-store'
online_store_df['Source'] = 'Online'

columns_to_keep = ['Full Name', 'Email', 'Phone', 'City', 'State', 'Total Spent', 'Total Orders', 'Source']
in_store_df = in_store_df[columns_to_keep]
online_store_df = online_store_df[columns_to_keep]

print("Concatenando DataFrames...")
combined_df = pd.concat([in_store_df, online_store_df], ignore_index=True)

combined_df['Email'] = combined_df['Email'].str.strip().str.lower()

print(f"Duplicados por Email antes de consolidar: {combined_df['Email'].duplicated().sum()}")

def consolidate_sources(group):
    if 'In-store' in group['Source'].values and 'Online' in group['Source'].values:
        source = 'Both'
    else:
        source = group['Source'].iloc[0] 
    return pd.Series({
        'Full Name': group['Full Name'].iloc[0],
        'Email': group['Email'].iloc[0],
        'Phone': group['Phone'].iloc[0],
        'City': group['City'].iloc[0],
        'State': group['State'].iloc[0],
        'Total Spent': group['Total Spent'].sum(),
        'Total Orders': group['Total Orders'].sum(),
        'Source': source
    })

print("Consolidando registros duplicados...")
consolidated_df = combined_df.groupby('Email', as_index=False).apply(consolidate_sources)

print(f"Tamaño después de consolidar: {consolidated_df.shape}")

print("Rellenando valores nulos...")
consolidated_df.fillna({'Phone': 'Not Provided'}, inplace=True)

# Guardar el DataFrame consolidado
print("Guardando datos en 'final_klaviyo_data.csv'...")
consolidated_df.to_csv('final_klaviyo_data.csv', index=False)

print("¡Proceso completado con éxito!")


Cargando archivos originales...
Tamaño de datos in-store: (5817, 11)
Tamaño de datos online: (31848, 20)
Renombrando columnas...
Concatenando DataFrames...
Duplicados por Email antes de consolidar: 5693
Consolidando registros duplicados...
Tamaño después de consolidar: (31971, 8)
Rellenando valores nulos...
Guardando datos en 'final_klaviyo_data.csv'...
¡Proceso completado con éxito!


/tmp/ipykernel_17132/3691082298.py:76: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  consolidated_df = combined_df.groupby('Email', as_index=False).apply(consolidate_sources)


In [4]:
import pandas as pd

data = pd.read_excel(r'/home/juandi/Documents/TOM_JETSON/Data/cleaned_data_for_klaviyo.xlsx')
data.to_csv('klaviyo_data_in_store1.csv')